# Homework 1b: Simple Systems, Complex Behaviors

organized by *Todd Gureckis, Brenden Lake, Alex Rich*  
class webpage: https://brendenlake.github.io/CCM-site/  
direct email to course instructors: instructors-ccm-spring2018@nyuccl.org

<div class="alert alert-danger" role="alert">
  This homework is due before midnight on Feb 6, 2018. 
</div>

---



One theme of the first lecture was the idea that even simple (cognitive) models can sometimes give rise to complex, non-intutive behaviors.  As a result, we often need computer simulations to help understand how even simple theories will play out.

For example, imagine a simple experiment where you roll two die and keep going until a particular pair comes up (say 7,11).  What is the distribution of the number of rolls needed to reach this outcome?  While we can certainly figure this out analytically, we might also make a simple program that simulates the process of rolling two die, and observe the results by having the computer run the game hundreds of thousands of times  (i.e., monte carlo method).  In this case, a *simulation* of the physical process of rolling dice can help us understand the behavior of the real system. 

Once we start trying to understand human cognitive processes or the structure of social behavior, we basically have to use simulations as there are unlikely to be simple, closed-form mathematical expressions that capture the phenomena.  

For simple systems the simulation approach is not particularly more interesting than doing the math.  However, in the examples we will consider in this part of the homework, sometimes what you get out of the simulation is more than you expect.  This can highlight the value of a simulation-based approach to modeling but also can raise our intuitions about how even simple models can quickly become hard to analyze.

<img src='images/complexification.png' width="600">

<div class="alert alert-info">
Computational complexity can also be beautiful to look at.  Check out <a href="http://www.complexification.net/">complexification.net</a>, a website devoted to computational explorations of complexity.  Figure above is examples of the <a href="http://www.complexification.net/gallery/machines/henonPhase/">henon phase</a>.
</div>

In this first homework, we will ‘play’ with two distinct systems based on simple rules, which are not only difficult to analyze analytically, but give rise to significantly more complex behavior at other levels of analysis than we might expect.  

The first is are what are known as **Cellular Automata** and the second is **Conway’s Game of Life**.  

The focus of the homework is simply to interact with each system to gain some intuition for how they behave, and a sense for how intuition can fail us sometimes in understanding even simple models.  This can also help to sharpen your Python programming skills.

---

<div class="alert alert-warning" role="alert">
  <strong>Warning!</strong> Before running other cells in this notebook you must first successfully execute the following cell which includes some libraries.
</div>

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.grid_objs as gro
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
import time
import pandas as pd
from IPython.display import display, Markdown


init_notebook_mode(connected=False)

# some helper functions we'll use later
from util import md_print, arrayplot, arrayplotbw,arrayplot_animate, neighborhoods

---

## Part I: Cellular Automata

One critical insight into the necessity of simulation and computational for understanding natural systems comes from the analysis of simple deterministic programs called [Cellular Automata](https://en.wikipedia.org/wiki/Cellular_automaton).

Consider first a single row of tiles which can have values either 0  or 1.

In [ ]:
start=np.array([[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]])
print (start)

We provided and imported above a simple function called `arrayplot` which lets you visualize such tile rows as a picture:

In [ ]:
arrayplotbw(start)

Now consider creating a new row underneath the first one the is a copy of the one above according to the following rules:  

<img src='images/rule254.png' width="300">

Where in the new row each cell takes on the value of depending on the cells immediately above it.  For instance (starting from the left) if all three cells centered over a particular new cell is 1 (black) then the new cell should be 1 (black).  The same is true for any configuration except the one of the left where if all the cells above are 0 (white) then the new cell is 0 (white).  This set of rules (there are eight of them) are completely deterministic in that nothing is left to chance.  You simply lookup in the rules what to do and create a new line. (For now on the edges just assume you keep the color as 0/white).

<div class="alert alert-success" role="alert">
  To verify you understanding, in the next cell try writing in the next row based on these row and use `arrayplotbw` to visualize the result.

</div>

In [ ]:
start=np.array(
    [
        [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
        [??]  # replace this line 
                                                     #with the correct application 
                                                     #of the above rules
    ])
arrayplot(start)

Instead of doing everything by hand we would like to write a computer program which will mindlessly apply these rules for us.  Here is one pretty straightfoward example using a function we provided called `neighborhoods()` which slices up the previous row into little subsets of three tiles at a time.

In [ ]:
start=np.array([[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]])
def rule(hood):
    if np.array_equal(np.array(hood),np.array([1,1,1])):
        return 1
    elif np.array_equal(np.array(hood),np.array([1,1,0])):
        return 1
    elif np.array_equal(np.array(hood),np.array([1,0,1])):
        return 1
    elif np.array_equal(np.array(hood),np.array([1,0,0])):
        return 1
    elif np.array_equal(np.array(hood),np.array([0,1,1])):
        return 1
    elif np.array_equal(np.array(hood),np.array([0,1,0])):
        return 1
    elif np.array_equal(np.array(hood),np.array([0,0,1])):
        return 1
    elif np.array_equal(np.array(hood),np.array([0,0,0])):
        return 0
    else:
        raise

def centerlist(n): # creates an array with n entries with a one in middle (rounds up)
    length = n+1 if n%2 == 0 else n
    tmp=[0]*length
    tmp[int(n/2)] = 1
    return tmp

NSTEPS = 30
start=[centerlist(NSTEPS*2)]
for i in range(NSTEPS):
    res = [rule(x) for x in neighborhoods(np.array(start[-1]),3,1)]
    res = [0]+res+[0] # adjust for the two edge cells
    start.append(res)
arrayplotbw(start)

<div class="alert alert-success" role="alert">
Now lets explore alternative rules.  Copy the cell above and modify it to represent the following rule system:

<img src='images/rule250.png' width="300">
    
What changed in the resulting picture?

</div>

In [ ]:
# put your modified code here

If you compare the two different sets up rules you can see that you can easily enumerate ALL the rules using $2^8$ bits (meaning there are 256 different sets of rules).  Let's make it easier to explore these different rules by making a more general way of applying the rules.

In [ ]:
def centerlist(n): # creates an array with n entries with a one in middle (rounds up)
    length = n+1 if n%2 == 0 else n
    tmp=[0]*length
    tmp[n/2] = 1
    return tmp

def elementaryrule(n): # determines the rule given a number (0-255)
    rule = [int(x) for x in bin(n)[2:]]
    return [0]*(8-len(rule))+rule

Using elementary rule we can now enumerate all possible rules.  For example the first rule we used above corresponds to:

In [ ]:
elementaryrule(254)

(from left to right all the configurations of previous tiles result in a 1 except the last one).  Likewise the second rule you consider corresponds to rule 250 (verify that the values output here line up with the left-to-right replacement rules):

In [ ]:
elementaryrule(250)

Given this, we can write a very simple function for applying the rules:

In [ ]:
def rotate(l, n):
    if len(l) == 0:
        return l
    n = n % len(l)
    return l[n:] + l[:n]

def CAStep(rule, a):
    t=rotate(a,1)+2*(np.array(a)+2*np.array(rotate(a,-1)))
    return [rule[7-x] for x in t]

# start with a 20 element list
NSTEPS = 80  # step foward 10
l=centerlist(NSTEPS*2)
rule = elementaryrule(254) # look up rule 254

history = [l]
for i in range(NSTEPS):
    res = CAStep(rule, history[-1])
    history.append(res)
    
arrayplotbw(history)

<div class="alert alert-success" role="alert">
Using the above code quickly play with `NSTEPS` and the rule number (e.g., `elementaryrule(254)`) to explore the implication of a wide range of rules.  Describe in a Markdown cell below what you notice using reference to particular rule numbers in your analysis.  Explore around and look for interesting patterns.  If you haven't tried it in the first few, be sure to explore rule 30.  Can you describe the pattern or detect any regularity?
</div>

<div class="alert alert-success" role="alert">
Select two of your favorite rules from your exploration above and consider what happens when you change the starting conditions to no longer be a single title "on" in the center of the list (up to you how you change it).  How does the evolution of the systems you considered change based on different initial configurations?  Do any patterns that seem regular and orderly become chaotic?  Do chaotic patterns become mundane and boring?
</div>

<div class="alert alert-success" role="alert">
Thinking about the range of behaviors you observed in your exploration, did the set of behaviors possible from this system surprise you?  Remember each situation (beside the previous problem) started with a single tile that was turned black and then through the iterative application of deterministic rules drew out interesting patterns and shapes. What general lessons can you draw from this exercise about making predictions about the behavior of simple systems?
</div>

It is interesting to compare the structure of some of these patterns that appear in nature for example here are examples of the patterns of snowflakes:
<img src='images/snowflakes.png' width="600">

Or the stripes on animals:
<img src='images/animalstripes.png' width="600">

It is possible that some of the endless forms that exist in nature also arise from rather simple chemical or genetic rules that makes many unique but still identifable patterns.

These final images (and much of the structure of this exercise) were taken from an interesting book by Steven Wolfram called "<a href="http://www.wolframscience.com/nks/">A New Kind of Science</a>" which is available free online.  Of particular note in this book is the wide range of interesting generative processes he explore including things like turing machine, network based automata, etc...  It is fun to read.

<img src='images/a-new-kind-of-science-cover.png' width="150">

---

---

## Part II: Conway's Game of Life

A system related to a Cellular Automata is Conway's Game of Life.  A computer scientist (John Conway) developed Conway’s Game of Life in the 1970s  while investigating simple rule systems.  The system starts with a two dimensional array of tiles.  On each time step the color of the tiles is updated according to a set of rules, usually involving the nearest neighbor of each tile (similar to the 1D cellular automata above).  The rules of the standard version of Conway's Game of Life are:

- Any cell with fewer than two “on” neighbors turns “off” (from loneliness)  
- Any cell with more than three “on” neighbors turns “off” (from crowding)  
- Any “on” cell with two or three live neighbors survives  
- Any “off” cell with 3 “on” neighbors turns “on” (comes back to life).

In this part of the homework, you are going to build up the necessary code to simulate Conway's Game of Life and then to explore the consequences of this system and sensitivity to initial conditions.

### Build a simulation of Conway's Game of Life

**Step 1: Set up initial "Stage"**

We first want to create a matrix of numbers which represents the current state of each tile in the world.  Let's imagine a $NxN$ world (where $N$ is some integer).  For now we should keep $N$ kind of small so that it doesn't overwhelm us.  First lets define a $3x3$ world where the edges are "on" and the center is "off":

In [ ]:
initial_config = [
                 [1, 1, 1 ],
                 [1, 0, 1 ],
                 [1, 1, 1 ]
                ]
np.array(initial_config) # prettify output using numpy

To help with the lab, we provide a simple piece of code <code>arrayplot()</code> that lets you visualize the current state of all the tiles as a picture.  The plot will be dark purple for any cell which has a value 1 and light purple for any cell which has a value 0:

In [ ]:
initial_config = [
                 [1, 1, 1 ],
                 [1, 0, 1 ],
                 [1, 1, 1 ]
                ]
arrayplot(initial_config)

**Step 2: Count neighbors**

Next, we need to count the number of neighbors that each tile in the grid has. You can do this a number of ways, but we will use `numpy` in order to help expose some of the features of that library.

First, it is useful to know about slices.  Using slices you can cut out small parts of a larger array.  For example if we start with a $10x10$ matrix of random integers between 0 and 9, we can selected out subpieces of the matrix.

In [ ]:
full_array = np.random.randint(0,10,(10,10))  # this is the full random array
print(full_array)

full_array[0:2,0:2]  # this gets the first two elements of each of the first two rows
full_array[0:-1,0:2] # this gets the first two elements of all rows except the last (-1)
full_array[2:-2,0:2] # this gets the first two elements of all the rows except the first two and last two

<div class="alert alert-success" role="alert">
  In the following cell:
  
  <ul>
  <li>Write the slice notiation to select the $4x4$ array which lies exactly at the center of the <code>full_array</code> array and print it.</li>
  
  <li>Write the slice notation to cut the $10x10$ array into four equal sized ($5x5$) quandrants.</li>
  </ul>
  
  Enter your response in a code cell immediately below this one and execute the cell to show the answer.
  
</div>

Each individual cell in a 2D array like this has 8 neighbors.  To implement Conway's game of life we need to count the number of neighbors which are alive or dead for each individual cell.  While we could do this with a bunch of <code>for</code> loops, it is useful to try to vectorize this operation.

The first observation we have is that it can be helpful to surround the entire board in a border of 1 tile wide zeros.  This way we don't have to do anything special with the tiles on the edge in terms of counting fewer neighbors (these cells have five or three legal neighbors).  To do this we will just write a simple function that adds a zero border.  Notice that this function uses slice notation to decide how to set elements in a new array.

In [ ]:
def add_zeros_border(Z):
    (x,y)=Z.shape
    newZ=np.zeros((x+2,y+2), int)
    newZ[1:-1,1:-1]=Z
    return newZ

In [ ]:
full_array = np.random.randint(0,10,(10,10))
add_zeros_border(full_array)

Once we do that we can find the neighbors of each cell in the main part of the matrix (not the border) by selecting sum matricies like this:

<img src='images/conway_matrix.png' width="300">

In this illustration the dark purple regions are our slice selection, the medium-light purple region is the "world stage" and the light purple is the board.  Now, consider the cell at the lower right bottom of the medium-light pink region (i.e., the stage).  We will call this cell "Bill".   The first selection <code>Z[:-2,:-2]</code> selects everything starting from the top left corder but stopping two rows or columns from the edge.  The lower right corner of this matrix then contains the upper left neighbor of "Bill".  Now consider the next illustration (clockwise) showing <code>Z[:-2,1:-1]</code>.  This selects a matrix the same as if we slid the main stage up by one row.  Once again the lower right corner of this selection contains the neighbor directly above "Bill".  Using the same logic you can see that across all the sliding around of this dark purple selection the lower right corner always contains one of the neighbors of "Bill".

<div class="alert alert-success" role="alert">
 
  Imagine a new cell named "Sally" which is one cell to the left of "Bill."  Using the same logic as above write in Markdown cell below which cell within each subselection is accounting for "Sally's" neighbors. (For Bill, it was the lower right cell.)  Which specific subselection slice is the one that actually grabs "Bill" (describe it using the <code>Z[X:X,X:X]</code> label)?  Enter your response in a markdown cell immediately below this one.
  
</div>

Using this approach we can now create a simple function that, given a numpy array with a border on it, will compute the number of neighbors for each cell.  

In [ ]:
initial_config = [
                 [1, 1, 1, 1, 1],
                 [1, 0, 0, 0, 1 ],
                 [1, 0, 0, 0, 1 ],
                 [1, 0, 0, 0, 1 ],
                 [1, 1, 1, 1, 1]
                ]
Z=np.array(initial_config)


def count_neighbors(Z):
        N = (Z[0:-2,0:-2] + Z[0:-2,1:-1] + Z[0:-2,2:] +
         Z[1:-1,0:-2]                + Z[1:-1,2:] +
         Z[2:  ,0:-2] + Z[2:  ,1:-1] + Z[2:  ,2:])
        return N

count_neighbors(add_zeros_border(Z))

**Step 3: Apply rules to get a new version of the world**

As a final step we need to apply the rules.  As a reminder:

- Any cell with fewer than two “on” neighbors turns “off” (from loneliness)  
- Any cell with more than three “on” neighbors turns “off” (from crowding)  
- Any “on” cell with two or three live neighbors survives  
- Any “off” cell with 3 “on” neighbors turns “on” (comes back to life).

We can apply these rules a logical operations directly on the numpy array:

In [ ]:
initial_config = [
                 [1, 1, 1, 1, 1],
                 [1, 0, 0, 0, 1 ],
                 [1, 0, 0, 0, 1 ],
                 [1, 0, 0, 0, 1 ],
                 [1, 1, 1, 1, 1]
                ]
Z = add_zeros_border(np.array(initial_config))
md_print("**Initial stage:**<hr>")
print(Z)
print("\n")

md_print("**Counting neighbors**<hr>")
N = count_neighbors(Z)
print(N)
print("\n")

md_print("**Who surivives? (True/False for each cell)**<hr>")
survive = ((N==2) | (N==3)) & (Z[1:-1,1:-1]==1) # any cell with two or three neighbords and is alive survives
print(survive)
print("\n")


md_print("**Who is born? (True/False for each cell)**<hr>")
birth = (N==3) & (Z[1:-1,1:-1]==0)  # any cell that has three neighbors and is curently off (exclude border)
print(birth)
print("\n")
Z[...] = 0  # everything else dies


#finally apply to get the new stage
Z[1:-1,1:-1][birth | survive] = 1 # apply the living rules

md_print("**Next stage:**<hr>")
print(Z)

<div class="alert alert-info">
Verify that you understand the logic behind each of these steps before moving on!  If you need additional information to help check out this website which was the basis of this implementation of Conway's game of life: http://www.labri.fr/perso/nrougier/teaching/numpy/numpy.html
</div>

**Step 4: Putting it all together!**

Now we have the all the necessary ingredients to run Conway's game of life:

In [ ]:
def add_zeros_border(Z):
    (x,y)=Z.shape
    newZ=np.zeros((x+2,y+2), int)
    newZ[1:-1,1:-1]=Z
    return newZ

def count_neighbors(Z):
        N = (Z[0:-2,0:-2] + Z[0:-2,1:-1] + Z[0:-2,2:] +
         Z[1:-1,0:-2]                + Z[1:-1,2:] +
         Z[2:  ,0:-2] + Z[2:  ,1:-1] + Z[2:  ,2:])
        return N

def step_conway(Z):
    N = count_neighbors(Z)
    
    # the conway 23/3 rules
    survive = ((N==2) | (N==3)) & (Z[1:-1,1:-1]==1) # any cell with two or three neighbords and is alive survives
    birth = (N==3) & (Z[1:-1,1:-1]==0)  # any cell that has three neighbors and is curently off (exclude border)
    Z[...] = 0  # everything else dies
    Z[1:-1,1:-1][birth | survive] = 1 # apply the living rules
    return Z

<div class="alert alert-success" role="alert">
The cell below show how to start with an initial configuration, plots the configuration, steps the conway rules forward by one step, and prints the results.
Design a different configuration (limiting yourself to stage sizes that are $10x10$) and run them for 10 steps plotting the results each time.  What do you notice?
</div>

In [ ]:
# extend this code to run with different initial configuration and to plot out the first 10 steps.
initial_config = [
                 [1, 1, 1, 1, 1],
                 [1, 0, 0, 0, 1 ],
                 [1, 0, 0, 0, 1 ],
                 [1, 0, 0, 0, 1 ],
                 [1, 1, 1, 1, 1]
                ]

stage = add_zeros_border(np.array(initial_config))
arrayplot(stage)
stage = step_conway(stage)
arrayplot(stage)

** Step 5: Animate! **

It is particularly useful to look at the sequence of these images as a movie or animation.   The following code uses some advanced plotting techniques to run a animation of each step of the game.

In [ ]:
NFRAMES = 15  # how many step to you want to take?
SKIP=1 # if you want to skip N frames make this > 1

# set you intiial configuration here
initial_config = np.random.randint(0,2,(20,20))  # set up a random configuration
Z=add_zeros_border(np.array(initial_config)) # add the zero border

# intialize animation
grid=pd.DataFrame(np.fliplr(np.transpose(Z)))
grid=grid.stack().reset_index()
grid.columns=['x','y','t0']


for i in range(NFRAMES):
    for j in range(SKIP):
        Z = step_conway(Z)  # iterate foward with the rules
    grid['t{}'.format(i+1)]=np.ravel(np.fliplr(np.transpose(Z))) # save results

arrayplot_animate(grid)  # display a animation with a time slider

### Explore the impact of initial conditions

<div class="alert alert-success" role="alert">
  Using the code cell below, explore the impact of differential initial configurations on the evolution of the system.  <br><br>
  
  First, run the cell block multiple times.  Because the `initial_config` is random to begin with each time you execute the cell you will get a different game.  What do you notice about the games?  Keep in mind that aside from the intial configuration everything is deterministic in the algorithm and follows very simple rules.  You can also experiment with different `NFRAMES` to allow the simulation to run longer (although note that very long simulations will take up a lot of memory for the animation).
  <br><br>
  
  Next, try "designing" at least two initial configurations on a $10x10$ stage.  Before running try to make a prediction about what will happen.  Will the result end in everything dead (off) or will it run chaotically?  For how long do you guess it will run before reaching a fixed state?  Show your initial configuration (use `arrayplot`) and write what your guess is about what happens .  Then write if you were correct or incorrect.
  
</div>

In [ ]:
# set you intiial configuration here
initial_config = np.random.randint(0,2,(20,20))  # set up a random configuration (place your configuration matrix here!)

In [ ]:
NFRAMES = 15  # how many step to you want to take?
SKIP=1 # if you want to skip N frames make this > 1

Z=add_zeros_border(np.array(initial_config)) # add the zero border

# intialize animation
grid=pd.DataFrame(np.fliplr(Z))
grid=grid.stack().reset_index()
grid.columns=['x','y','t0']


for i in range(NFRAMES):
    for j in range(SKIP):
        Z = step_conway(Z)  # iterate foward with the rules
    grid['t{}'.format(i+1)]=np.ravel(np.fliplr(Z)) # save results

arrayplot_animate(grid)  # display a animation with a time slider

### Introduction to the ecology of the game

<div class="alert alert-success" role="alert">
  For each of the configurations below run the animation for 50 time steps and write down in a Markdown cell what you observe.

</div>

**Configuration 1**

In [ ]:
initial_config = np.zeros((10,10),int)
a1 = np.array([ [0,1,0],
  [0,1,0],
  [0,1,0]])
a2 = np.array([ [0,0,0],
  [1,1,1],
  [0,0,0]])
initial_config[7:10,7:10] = a1
initial_config[0:3,7:10] = a2
initial_config[0:3,0:3] = a1
initial_config[7:10,0:3] = a2

<div class="alert alert-success" role="alert">
What did you observe?  How did it compare to your expectations?  Is there anything exceptional or different about this configuration?
</div>

**Configuration 2**

In [ ]:
initial_config = np.zeros((10,10),int)
a1 = np.array([ [1,1,1],
  [1,0,0],
  [0,1,0]])
initial_config[7:10,7:10] = a1

<div class="alert alert-success" role="alert">
What did you observe?  How did it compare to your expectations?  Is there anything exceptional or different about this configuration?
</div>

**Configuration 3**

In [ ]:
initial_config = np.zeros((20,20),int)
a1 = np.array([ [0,1,1,1],
  [1,0,0,1],
  [0,0,0,1],
  [0,0,0,1],
  [1,0,1,0] ])

initial_config[15:20,3:7] = a1

<div class="alert alert-success" role="alert">
What did you observe?  How did it compare to your expectations?  Is there anything exceptional or different about this configuration?
</div>

**Configuration 4**

Now create a large board, about $30x30$ and enter this pattern by hand so that is somewhat centered by near the upper half of the grid.

<img src='images/glider_gun.png' width="400">

<div class="alert alert-success" role="alert">
What did you observe when you ran this configuration?  How did it compare to your expectations?  Is there anything exceptional or different about this configuration?  How does this one compare to the previous ones?
</div>

### Emergence

**Emergence** is a concept by which some larger pattern or form comes into being through the interactions for smaller and simpler things.  Conway's Game of Life is a particularly unique case because the patterns the emerge and that you described above are sometimes call "artificial life" because the hang to together as patterns we recognize as almost being "alive".  Out of the primodial computational soup of the game rules, we find creatures which crawl around, bang into one another, exist as stable pattern for eternity and which maybe even "birth" new creatures.  This highlight just another way in which simple systems can give rise to patterns of complexity which are very hard to predict and hard to describe analytically.

<img src="images/flock.jpg" width="300">

These are ideas we should keep in mind for later lectures when we begin talking about neural network and the computing properties of systems compose of million or billions of simple interacting units (e.g., neurons!)

<img src="images/neurons.jpeg" width="300">

## Turning in homeworks

When you are finished with this notebook. Save your work in order to turn it in.  To do this select *File*->*Download As...*->*HTML*.

<img src="images/save-pdf.png" width="300">

You can turn in your assignments using NYU Classes webpage for the course (available on https://home.nyu.edu).